<a href="https://colab.research.google.com/github/dipakamiitk/Crosslingual-WSD/blob/master/XLMR_finetuning_MCL_MN_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# import pandas
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
! ls "/content/drive/My Drive/datasets/Split_WiC_dataset"

Cosine_Sim  dev  train


In [ ]:
# Load the mcl en data
train_mcl_df = pd.read_csv("/content/drive/MyDrive/datasets/MCL_WiC_old_format/new_train_format_english_en.csv")
print(train_mcl_df)

                       id  ... tag
0        training.en-en.0  ...   F
1        training.en-en.1  ...   F
2        training.en-en.2  ...   T
3        training.en-en.3  ...   T
4        training.en-en.4  ...   T
...                   ...  ...  ..
7995  training.en-en.7995  ...   T
7996  training.en-en.7996  ...   F
7997  training.en-en.7997  ...   T
7998  training.en-en.7998  ...   F
7999  training.en-en.7999  ...   T

[8000 rows x 12 columns]


In [ ]:
# load OOV split data for training
mix_df = pd.read_csv("/content/drive/MyDrive/datasets/MCL_WiC_old_format/split_dev_seq/train_combined_7.2k")
mix_df.head()

,id,lemma,pos,position1,start1,end1,position2,start2,end2,sent1,sent2,tag
0,dev.ar-ar.99,فَحَصَ,VERB,0,1,5,6,32,38,ويفحص التقرير الثاني، وهو استعراض لقدرات استجا...,كما قضت، برضا صاحبة الشكوى، بأن يفحصها في عياد...,F
1,dev.ar-ar.100,نِظامِيٌّ,ADJ,9,52,58,6,36,42,ومعظم هؤلاء العمال من الأميين وليس لديهم علاقا...,والمهاجرون الذين يوجدون في حالة غير نظامية هم ...,F
2,dev.ar-ar.101,نِظامِيٌّ,ADJ,9,52,58,11,77,83,ومعظم هؤلاء العمال من الأميين وليس لديهم علاقا...,يجب تحديد، جمع، تأكيد وتقاسم المعلومات الخاصة ...,F
3,dev.ar-ar.103,ضَرَبَ,VERB,3,13,16,10,58,62,كما يدعي أنه ضرب في حضور مدعية عندما أنكر اعتر...,وبعد ذلك استل أحد ضابطي الشرطة هراوته وأخذ، حس...,T
4,dev.ar-ar.104,حَطَّ,VERB,16,95,98,18,109,112,ومع ذلك تواصل الأمم المتحدة عرقلة نفسها بهيكل ...,وأفادت التقارير بأن الآلة الطائرة التي هي بدون...,F


Load Xl-WiC datasets

In [ ]:
xl_zh_df = pd.read_csv("/content/drive/MyDrive/datasets/XL-WIC/xlwic_zh_val_deepak_format.csv")
xl_zh_df.head()

,lemma,pos,start1,end1,start2,end2,sent1,sent2,tag,position1,position2
0,轉,V,9,10,44,45,為什麼我會用情至深轉不出情關，別人是在情感路上徘徊，而我是在親情的深淵裡沉淪。,他其實也明白自己的個性，常常會因一句話或一件事，被帶入死胡同裡，ㄧ直往死胡同裡鑽，然後又轉不出來。,T,0,0
1,打,V,12,13,11,12,莫研仰脖叫道：「你倆打屁打得還真沒完沒了地哇，我不等了，要先去紅珊姐那裡了。」,沒多久就看到顏妤佑打屁打到一半突然想到有社團表演，就帶著他們衝去活動中心看康輔的表演。,T,0,0
2,轉,V,9,10,11,12,凝重的天空一會兒突轉清朗，北風趕走濃雲，湖的上空出現了一片藍天。,華輝猜知了她的心意，語轉溫和，說道：「李姑娘，你只須助我拔出毒針，我要給你許許多多金銀珠寶。」,T,0,0
3,發出,V,13,15,37,39,二小時後他手上的求救器自動發出求救訊息，消防局監測人員接獲訊號後派救護車緊急將他送醫。,這是指利用儀器連續偵測、記錄環境中毒性化學物質濃度，當濃度超過設定值時，可發出警報訊號之設備。,T,0,0
4,上,V,17,18,15,16,她目睹兒子判若兩人的改變，毅然把才上小學的女兒也轉送到森小。,她強調自己也是平凡的大學生，剛上大一時，雖曾因這較特別的工作招來一些異樣眼光。,T,0,0


In [ ]:
xl_nl_df = pd.read_csv("/content/drive/MyDrive/datasets/XL-WIC/xlwic_dutch_nl_val_deepak_format.csv")
xl_nl_df.head()

,lemma,pos,start1,end1,start2,end2,sent1,sent2,tag,position1,position2
0,pijn,N,7,11,14,18,ik heb pijn in mijn hoofd/aan mijn schouder.,waar doet het pijn?,T,2,3
1,pijn,N,20,24,8,12,heb je wat tegen de pijn?,ik voel pijn in mijn linkerknie.,T,5,2
2,periode,N,3,10,24,31,de periode 1100 - 1300 wordt gekenmerkt door [ ].,ze is verkozen voor een periode van twee jaar.,T,1,5
3,pijn,N,14,18,19,23,waar doet het pijn?,een medicijn om de pijn te stillen.,T,3,4
4,periode,N,31,38,4,11,het akkoord dateert nog uit de periode Lubbers-I.,een periode van bezinning/strenge vorst/schaar...,T,6,1


In [ ]:
xl_hr_df = pd.read_csv("/content/drive/MyDrive/datasets/XL-WIC/xlwic_croation_hr_val_deepak_format.csv")
xl_hr_df.head()

,lemma,pos,start1,end1,start2,end2,sent1,sent2,tag,position1,position2
0,raditi,V,12,18,0,6,Naporno sam radila da bih diplomirala na vrijeme.,Radila je na poboljšanju životnih uvjeta u Afr...,T,2,0
1,traka,N,20,25,36,41,Na toj su pokretnoj traci radila tri radnika.,Cijele je noći radila u tvornici na traci.,T,4,7
2,boja,N,51,57,7,11,Učenici su dobili za zadaću nacrtati sliku vod...,Uljene boje omiljena su tehnika impresionistič...,T,8,1
3,lik,N,50,56,50,58,Izračunavali su površine različitih geometrijs...,Pravokutnik i kvadrat pripadaju dvodimenzional...,T,5,5
4,orma,N,62,66,29,33,Ljepota konja dodatno se ističe kada ga se zao...,Tek s pronalaskom primjerene orme postalo je m...,T,11,4


In [ ]:
xl_da_df = pd.read_csv("/content/drive/MyDrive/datasets/XL-WIC/xlwic_danish_da_val_deepak_format.csv")
xl_da_df.head()

,lemma,pos,start1,end1,start2,end2,sent1,sent2,tag,position1,position2
0,gratisavis,N,32,42,21,31,fremtiden er den annoncebetalte gratisavis.,Den husstandsomdelte gratisavis Dato taber ifø...,T,4,2
1,google,V,40,47,68,75,"Man søger ikke efter noget længere, man google...",en stadig voksende del af befolkningen sidder ...,T,7,11
2,indie,N,91,96,55,60,Jyske troubadurer er i København omtrent lige ...,[gruppen] epo-555 kan om nogen leve op til bet...,T,14,9
3,bringe,V,15,21,17,23,Tanken om Leon bragte blodet op i hendes kinder.,Et års nabostrid bragte mig i går på skadestue...,T,3,3
4,gratisavis,N,44,56,15,27,det [er da] et udmærket initiativ med de to gr...,Abbonenter som gratisavisen Weekend Fyn kunne ...,T,9,2


In [ ]:
# combine the datasets
train_df = pd.concat([train_mcl_df, mix_df
                      , xl_da_df, xl_hr_df, xl_nl_df, xl_zh_df
                      # , farsi_df
                      ], ignore_index=True)
print(train_df)
train_df.head()

                     id  ... tag
0      training.en-en.0  ...   F
1      training.en-en.1  ...   F
2      training.en-en.2  ...   T
3      training.en-en.3  ...   T
4      training.en-en.4  ...   T
...                 ...  ...  ..
19447               NaN  ...   F
19448               NaN  ...   F
19449               NaN  ...   F
19450               NaN  ...   F
19451               NaN  ...   F

[19452 rows x 12 columns]


,id,lemma,pos,position1,start1,end1,position2,start2,end2,sent1,sent2,tag
0,training.en-en.0,play,NOUN,11,69,73,2,10,14,In that context of coordination and integratio...,A musical play on the same subject was also st...,F
1,training.en-en.1,play,NOUN,11,69,73,22,112,116,In that context of coordination and integratio...,"In schools, when water is needed, it is girls ...",F
2,training.en-en.2,esteem,NOUN,7,33,39,19,106,112,We would also like to convey our esteem and co...,"Father Lini said that, because of that, the Un...",T
3,training.en-en.3,esteem,NOUN,7,33,39,4,20,26,We would also like to convey our esteem and co...,This attests to the esteem and trust enjoyed b...,T
4,training.en-en.4,holder,NOUN,12,74,81,6,27,33,This growth is the direct result of the increa...,A person may be either the holder of an option...,T


In [ ]:
dev_df_en =pd.read_csv("/content/drive/MyDrive/datasets/MCL-WiC/dev_data.csv").rename(columns={"sentence1":"sent1", "sentence2":"sent2"})
print(dev_df_en)
dev_df_en.head()

                id         lemma   pos  ... start2 end2  tag
0      dev.en-en.0      superior  NOUN  ...     41   50    F
1      dev.en-en.1      superior  NOUN  ...     44   53    T
2      dev.en-en.2  acquaintance  NOUN  ...     41   54    F
3      dev.en-en.3  acquaintance  NOUN  ...     74   86    F
4      dev.en-en.4       baggage  NOUN  ...      6   13    T
..             ...           ...   ...  ...    ...  ...  ...
995  dev.en-en.995         crash  NOUN  ...     75   80    T
996  dev.en-en.996     calculate  VERB  ...     12   22    F
997  dev.en-en.997     calculate  VERB  ...     11   21    F
998  dev.en-en.998         click  VERB  ...     92   98    T
999  dev.en-en.999         click  VERB  ...     59   66    F

[1000 rows x 10 columns]


,id,lemma,pos,sent1,sent2,start1,end1,start2,end2,tag
0,dev.en-en.0,superior,NOUN,No clause in a contract shall be interpreted a...,While fully aware that bishops and major super...,78,87,41,50,F
1,dev.en-en.1,superior,NOUN,No clause in a contract shall be interpreted a...,"In Senegal too, the customs officer and his su...",78,87,44,53,T
2,dev.en-en.2,acquaintance,NOUN,Such acquaintance is a right and not an obliga...,The complaints tend to be lodged against acqua...,5,17,41,54,F
3,dev.en-en.3,acquaintance,NOUN,Such acquaintance is a right and not an obliga...,Sexual violence by non-partners refers to viol...,5,17,74,86,F
4,dev.en-en.4,baggage,NOUN,Where any baggage of any passenger contains fi...,In my baggage I had a Hungarian grammar book a...,10,17,6,13,T


In [ ]:
dev_df_mix = pd.read_csv("/content/drive/MyDrive/datasets/MCL_WiC_old_format/split_dev_seq/dev_combined_400x2").rename(columns={"sentence1":"sent1", "sentence2":"sent2"})
print(dev_df_mix)
dev_df_mix.head()

                id  ... tag
0      dev.ar-ar.0  ...   T
1      dev.ar-ar.1  ...   T
2      dev.ar-ar.2  ...   T
3      dev.ar-ar.3  ...   T
4      dev.ar-ar.4  ...   F
..             ...  ...  ..
795   dev.zh-zh.95  ...   T
796   dev.zh-zh.96  ...   F
797   dev.zh-zh.97  ...   F
798   dev.zh-zh.98  ...   F
799  dev.zh-zh.102  ...   F

[800 rows x 12 columns]


,id,lemma,pos,position1,start1,end1,position2,start2,end2,sent1,sent2,tag
0,dev.ar-ar.0,مَلاك,NOUN,10,62,66,11,66,70,ونظرا لأهمية هذه المسائل لسير عمل المحكمة مستق...,ولا توجد حراسة أمام جميع البعثات الدبلوماسية ب...,T
1,dev.ar-ar.1,مَلاك,NOUN,10,62,66,11,67,74,ونظرا لأهمية هذه المسائل لسير عمل المحكمة مستق...,وأعربت عن رغبتها في الحصول على معلومات بشأن مو...,T
2,dev.ar-ar.2,فَوضَى,NOUN,11,63,67,9,53,57,ويؤدي هذا المرض، الذي ينتشر بين أكبر قطاعات ال...,والواقع أن آلية نزع السلاح المتعددة الأطراف تع...,T
3,dev.ar-ar.3,فَوضَى,NOUN,11,63,67,24,129,133,ويؤدي هذا المرض، الذي ينتشر بين أكبر قطاعات ال...,وفي مقابل ذلك، فإنه في حالة تعذر حل هذه المشكل...,T
4,dev.ar-ar.4,خَطِيئَة,NOUN,7,43,50,14,82,87,والتشريع ذات الأولوية هو إعادة هيكلة ضرائب الخ...,وتم استحداث هذا الحل كخيار متاح للكاثوليكيين ا...,F


In [ ]:
dev_df = pd.concat([dev_df_mix, dev_df_en], ignore_index=True)
print(dev_df)

                 id  ... tag
0       dev.ar-ar.0  ...   T
1       dev.ar-ar.1  ...   T
2       dev.ar-ar.2  ...   T
3       dev.ar-ar.3  ...   T
4       dev.ar-ar.4  ...   F
...             ...  ...  ..
1795  dev.en-en.995  ...   T
1796  dev.en-en.996  ...   F
1797  dev.en-en.997  ...   F
1798  dev.en-en.998  ...   T
1799  dev.en-en.999  ...   F

[1800 rows x 12 columns]


In [ ]:
test_mix = pd.read_csv("/content/drive/MyDrive/datasets/MCL_WiC_old_format/Test/multi/multi_test.csv").rename(columns={"sentence1":"sent1", "sentence2":"sent2"})
print(test_mix[['sent1']])
test_mix.head()


                                                  sent1
0     وكان ستون من مراقبي الشرطة هؤﻻء قد خدموا في اﻷ...
1     وكان ستون من مراقبي الشرطة هؤﻻء قد خدموا في اﻷ...
2     وناقشت أفرقة خبراء متعددة القوانين عبر اﻹقليمي...
3     وناقشت أفرقة خبراء متعددة القوانين عبر اﻹقليمي...
4     ويرى الوزير أنه لا يزال يتعين على أفراد الشعب ...
...                                                 ...
3995  Лидерство – феномен власти, способность одного...
3996  Над Корсаком нависает угроза ареста, и он, пер...
3997  Над Корсаком нависает угроза ареста, и он, пер...
3998  Кроме того, если язык отделился от праязыка до...
3999  Кроме того, если язык отделился от праязыка до...

[4000 rows x 1 columns]


,id,lemma,pos,position1,start1,end1,position2,start2,end2,sent1,sent2
0,test.ar-ar.0,خَدَمَ,VERB,7,35,40,17,95,100,وكان ستون من مراقبي الشرطة هؤﻻء قد خدموا في اﻷ...,وآخر فئة من الموظفين ينظر في أمر انهاء خدماتهم...
1,test.ar-ar.1,خَدَمَ,VERB,7,35,40,8,47,53,وكان ستون من مراقبي الشرطة هؤﻻء قد خدموا في اﻷ...,وكما ورد في تعليقنا السابق، معظم اﻷطفال الجنود...
2,test.ar-ar.2,نَاقَشَ,VERB,0,0,6,0,0,5,وناقشت أفرقة خبراء متعددة القوانين عبر اﻹقليمي...,وناقش المشتركون في اﻻجتماع ١١ ورقة عمل، واستعر...
3,test.ar-ar.3,نَاقَشَ,VERB,0,0,6,6,26,31,وناقشت أفرقة خبراء متعددة القوانين عبر اﻹقليمي...,ووافق على أنه من اﻷفضل أن تناقش هذه المادة، مع...
4,test.ar-ar.4,فَهِمَ,VERB,11,58,64,2,9,13,ويرى الوزير أنه لا يزال يتعين على أفراد الشعب ...,وقال إنه يفهم أن اللجنة ترغب في اتباع اﻹجراء ذ...


In [ ]:
test_df = pd.concat([test_mix], ignore_index=True)
print(test_df)

                  id  ...                                              sent2
0       test.ar-ar.0  ...  وآخر فئة من الموظفين ينظر في أمر انهاء خدماتهم...
1       test.ar-ar.1  ...  وكما ورد في تعليقنا السابق، معظم اﻷطفال الجنود...
2       test.ar-ar.2  ...  وناقش المشتركون في اﻻجتماع ١١ ورقة عمل، واستعر...
3       test.ar-ar.3  ...  ووافق على أنه من اﻷفضل أن تناقش هذه المادة، مع...
4       test.ar-ar.4  ...  وقال إنه يفهم أن اللجنة ترغب في اتباع اﻹجراء ذ...
...              ...  ...                                                ...
3995  test.ru-ru.995  ...  Обычно событие заставляет человека задуматься ...
3996  test.ru-ru.996  ...  Над дном в виде шатра нависает крыша четвёртог...
3997  test.ru-ru.997  ...  Над залом нависает гигантский купол с деревянн...
3998  test.ru-ru.998  ...  Человеку давно были известны простейшие кузнеч...
3999  test.ru-ru.999  ...  Не так давно этот маленький город приносил око...

[4000 rows x 11 columns]


In [ ]:
test_df.shape

(4000, 11)

In [ ]:
!nvidia-smi

Fri Jan 29 08:56:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    22W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 1.8MB 4.2MB/s 
     |████████████████████████████████| 2.9MB 29.3MB/s 
     |████████████████████████████████| 890kB 30.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c69e34bbab3749dab29979bc091a6edd3f5f42525c1c9d7ff25396cb2e4686ac
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 4.1MB/s 


In [ ]:
from transformers import XLMRobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-large')

Loading roberta tokenizer...


In [ ]:
# sentences is a list 0f str

# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []

test_sentences_1 = []
test_sentences_2 = []

list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])

list_test_sentences_1 =  list(test_df['sent1'])
list_test_sentences_2 = list(test_df['sent2'])

for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'" '+sentence_1[s1:e1]+' "'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'" '+sentence_2[s2:e2]+' "'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'" '+d_sentence_1[s1:e1]+' "'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)
  if i == 8:
    ars = d_sentence_1

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'" '+d_sentence_2[s2:e2]+' "'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)

for i in range(len(list_test_sentences_1)):
  d_sentence_1 = list_test_sentences_1[i]
  s1 = int(test_df['start1'][i])
  e1 = int(test_df['end1'][i])
  # print(d_sentence_1)
  d_sentence_1 = d_sentence_1[:s1]+'" '+d_sentence_1[s1:e1]+' "'+ d_sentence_1[e1:]
  test_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_test_sentences_2[i]
  s2 = int(test_df['start2'][i])
  e2 = int(test_df['end2'][i])
  # if i==2:
  #   print((s2,e2))
  #   print(d_sentence_2)
  #   print(d_sentence_2[:s2])
  #   print(d_sentence_2[s2:e2])
  #   print(d_sentence_2[e2:])
  #   # print(d_sentence_2[:s2]+'a '+d_sentence_2[s2:e2]+' b'+ d_sentence_2[e2:])
  #   # print(d_sentence_2[e2:]+'" '+d_sentence_2[s2:e2]+' "'+ d_sentence_2[:s2])
  #   print(d_sentence_2[:s2]+'"'+" "+d_sentence_2[s2:e2]+" "+'"'+ d_sentence_2[e2:])
  #   # print(d_sentence_2[e2:]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[:s2])
  #   print()

  d_sentence_2 = d_sentence_2[:s2]+'" '+d_sentence_2[s2:e2]+' "'+ d_sentence_2[e2:]
  test_sentences_2.append(d_sentence_2)

  # print((dev_sentences_1[i], dev_sentences_2[i]))

print((train_sentences_2[:10]))
print((train_sentences_1[:10]))
print(dev_sentences_1[:10])
print(dev_sentences_2[:10])
print(test_sentences_1[:10])
print(test_sentences_2[:10])

['A musical " play " on the same subject was also staged in Kathmandu for three days.', 'In schools, when water is needed, it is girls who are sent to fetch it, taking time away from their studies and " play ".', 'Father Lini said that, because of that, the United Nations has a very special place in the affections and " esteem " of the people of Vanuatu.', 'This attests to the " esteem " and trust enjoyed by your country.', 'A person may be either the " holder " of an option, being the person entitled to buy or sell; or the writer of the option, being the person required to honour the holder’s right to buy or sell.', 'Over 5,000 now hold legal immigrant documents, which, after five years of annual renewal, entitles the " holder " to apply for permanent residence.', 'The conclusion of the first " reading " would make it possible to begin negotiations on a streamlined text.', 'It was precisely that consideration which led the Commission on first " reading " to introduce a special regime 

In [ ]:
encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_train['input_ids'][0]))
print(type(encoded_inputs_train['input_ids']))


encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

encoded_inputs_test = tokenizer(test_sentences_1, test_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

print(test_sentences_1[2])
print(test_sentences_2[2])
print((encoded_inputs_test['offset_mapping'][2]))
print((encoded_inputs_test['input_ids'][2]))

250
<class 'torch.Tensor'>
176
<class 'torch.Tensor'>
" وناقشت " أفرقة خبراء متعددة القوانين عبر اﻹقليمية والقوانين المتصلة بالمواد اﻹباحية المستغل فيها اﻷطفال.
" وناقش " المشتركون في اﻻجتماع ١١ ورقة عمل، واستعرضوا التقدم المحرز في التعاون في ميدان التجارة والتنمية ووضعوا عدة توصيات من أجل اتخاذ إجراءات مستقبلية.
tensor([[  0,   0],
        [  0,   1],
        [  2,   3],
        [  3,   7],
        [  7,   8],
        [  9,  10],
        [ 11,  13],
        [ 13,  16],
        [ 17,  20],
        [ 20,  22],
        [ 23,  29],
        [ 30,  38],
        [ 39,  42],
        [ 43,  49],
        [ 49,  50],
        [ 51,  55],
        [ 55,  58],
        [ 58,  60],
        [ 61,  65],
        [ 65,  68],
        [ 69,  73],
        [ 73,  76],
        [ 77,  78],
        [ 78,  81],
        [ 81,  83],
        [ 84,  89],
        [ 89,  91],
        [ 92,  96],
        [ 97, 102],
        [102, 103],
        [  0,   0],
        [  0,   0],
        [  0,   1],
        [  2,   3],
     

In [ ]:
train_sentences_1[800]

'The " disintegration " of rigid systems like those of the Soviet Union and the former Yugoslavia has produced a number of new States and new Members of the United Nations.'

In [ ]:
def check_end(offsets, i, e):
  d = offsets[i][0]

  for j in range(i, len(offsets)):
    a = offsets[j][0]
    b = offsets[j][1]

    if (b==e):
      return True
    
    if (a == d):
      d = b
      continue
    else:
      return False
  return False


In [ ]:
# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])
  e1 = int(train_df['end1'][j])
  e2 = int(train_df['end2'][j])
  pos1 = -1
  pos2 = -1
  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1+1 and second_sent == False:
      if check_end(offsets, i, e1+1+1):
        pos1 = i
    elif offset[0] == s2+1+1 and second_sent == True:
      if check_end(offsets, i, e2+1+1):
        pos2 = i
        break
  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0 and not offsets[i-1][1]==0:
        second_sent = True

      if offset[0] == s1+1 and second_sent == False:
        if check_end(offsets, i, e1+1):
          pos1 = i
      elif offset[0] == s2+1 and second_sent == True:
        if check_end(offsets, i, e2+1):
          pos2 = i
          break
  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1 and second_sent == False:
        if check_end(offsets, i, e1):
          pos1 = i
      elif offset[0] == s2 and second_sent == True:
        if check_end(offsets, i, e2):
          pos2 = i
          break
  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1-1 and second_sent == False:
        if check_end(offsets, i, e1-1):
          pos1 = i
      elif offset[0] == s2-1 and second_sent == True:
        if check_end(offsets, i, e2-1):
          pos2 = i
          break

  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1-2 and second_sent == False:
        if check_end(offsets, i, e1-2):
          pos1 = i
      elif offset[0] == s2-2 and second_sent == True:
        if check_end(offsets, i, e2-2):
          pos2 = i
          break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])
  e1 = int(dev_df['end1'][j])
  e2 = int(dev_df['end2'][j])
  pos1 = -1
  pos2 = -1
  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1+1 and second_sent == False:
      if check_end(offsets, i, e1+1+1):
        pos1 = i
    elif offset[0] == s2+1+1 and second_sent == True:
      if check_end(offsets, i, e2+1+1):
        pos2 = i
        break
  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1+1 and second_sent == False:
        if check_end(offsets, i, e1+1):
          pos1 = i
      elif offset[0] == s2+1 and second_sent == True:
        if check_end(offsets, i, e2+1):
          pos2 = i
          break
  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1 and second_sent == False:
        if check_end(offsets, i, e1):
          pos1 = i
      elif offset[0] == s2 and second_sent == True:
        if check_end(offsets, i, e2):
          pos2 = i
          break
  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1-1 and second_sent == False:
        if check_end(offsets, i, e1-1):
          pos1 = i
      elif offset[0] == s2-1 and second_sent == True:
        if check_end(offsets, i, e2-1):
          pos2 = i
          break

  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1-2 and second_sent == False:
        if check_end(offsets, i, e1-2):
          pos1 = i
      elif offset[0] == s2-2 and second_sent == True:
        if check_end(offsets, i, e2-2):
          pos2 = i
          break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)

test_pos_1 = []
test_pos_2 = []

for j in range(len(test_sentences_1)):
  offsets = encoded_inputs_test['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(test_df['start1'][j])
  s2 = int(test_df['start2'][j])
  e1 = int(test_df['end1'][j])
  e2 = int(test_df['end2'][j])
  pos1 = -1
  pos2 = -1
  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1+1 and second_sent == False:
      if check_end(offsets, i, e1+1+1):
        pos1 = i
    elif offset[0] == s2+1+1 and second_sent == True:
      if check_end(offsets, i, e2+1+1):
        pos2 = i
        break
  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1+1 and second_sent == False:
        if check_end(offsets, i, e1+1):
          pos1 = i
      elif offset[0] == s2+1 and second_sent == True:
        if check_end(offsets, i, e2+1):
          pos2 = i
          break
  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1 and second_sent == False:
        if check_end(offsets, i, e1):
          pos1 = i
      elif offset[0] == s2 and second_sent == True:
        if check_end(offsets, i, e2):
          pos2 = i
          break
  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1-1 and second_sent == False:
        if check_end(offsets, i, e1-1):
          pos1 = i
      elif offset[0] == s2-1 and second_sent == True:
        if check_end(offsets, i, e2-1):
          pos2 = i
          break

  if pos1 == -1 or pos2 ==-1:
    second_sent = False
    for i, offset in enumerate(offsets):
      if i == 0:
        continue #cls
      if offset[1] == 0:
        second_sent = True

      if offset[0] == s1-2 and second_sent == False:
        if check_end(offsets, i, e1-2):
          pos1 = i
      elif offset[0] == s2-2 and second_sent == True:
        if check_end(offsets, i, e2-2):
          pos2 = i
          break
  test_pos_1.append(pos1)
  test_pos_2.append(pos2)
print(dev_pos_1[:100])
print(dev_pos_2[800:810])
print(test_pos_1[:10])
print(len(test_pos_2))

train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

test_pos_1 = torch.LongTensor(test_pos_1)
test_pos_2 = torch.LongTensor(test_pos_2)

test_pos = torch.stack((test_pos_1, test_pos_2), dim=1)

[20, 20, 21, 21, 17, 17, 18, 18, 3, 1, 21, 21, 8, 8, 25, 25, 19, 19, 62, 62, 8, 8, 4, 4, 7, 7, 6, 6, 3, 3, 53, 53, 32, 32, 8, 8, 3, 3, 3, 3, 4, 4, 9, 9, 3, 3, 3, 3, 8, 8, 12, 12, 12, 12, 4, 4, 3, 3, 3, 3, 6, 6, 11, 11, 8, 8, 22, 22, 50, 50, 10, 10, 6, 6, 31, 31, 9, 9, 3, 3, 6, 6, 17, 17, 4, 4, 7, 7, 9, 9, 4, 4, 7, 7, 7, 7, 3, 3, 3, 6]
[37, 38, 32, 40, 43, 68, 46, 48, 44, 53]
[11, 11, 2, 2, 18, 18, 6, 6, 16, 16]
4000


In [ ]:
# labels = torch.from_numpy(train_gold_df['label'].values)
train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)
print(train_labels)

dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)

# test_gold_df_tmp = test_df[['tag']].replace({'F' : 0, 'T' : 1})
# test_labels = torch.from_numpy(test_gold_df_tmp.values)
# print(len(test_labels))

tensor([[0],
        [0],
        [1],
        ...,
        [0],
        [0],
        [0]])


In [ ]:
from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)

test_dataset = TensorDataset(encoded_inputs_test['input_ids'],
                              encoded_inputs_test['attention_mask'], test_pos
                            #  , test_labels
                             )


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

**Models and training**

In [ ]:
from transformers import XLMRobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "xlm-roberta-large"
        hidden_states = False
        self.encoder = XLMRobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer

In [ ]:
class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

In [ ]:
model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)

In [ ]:
optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )

# scheduler??


In [ ]:
def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)

In [ ]:
def cross_entropy(predictions, targets):
    N = predictions.shape[0]
    predictions = 1/(1+np.exp(-predictions))
    ce = -np.sum(targets * np.log(predictions) + (1-targets) * np.log(1-predictions)) / N
    return ce

def cross_entropy_avg(predictions, targets,m):
    N = predictions.shape[0]
    predictions = 1/(1+np.exp(-predictions))
    predictions = (predictions[:m]+predictions[m:])/2
    ce = -np.sum(targets * np.log(predictions) + (1-targets) * np.log(1-predictions)) / N
    return ce

In [ ]:
def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat.shape)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# path_en = "/content/drive/MyDrive/datasets/final/models/English/xlmr_mix_no_en_1"
path_mix = "/content/drive/MyDrive/SemEval Models b/Mix/Seq/xlmr_spaced_mcl_xlwic_7.2k_mix_seq_run_2"
path_ar = "/content/drive/MyDrive/SemEval Models b/Arabic/xlmr_spaced_mcl_xlwic_7.2k_mix_seq_run_2"
path_en = "/content/drive/MyDrive/SemEval Models b/English/xlmr_spaced_mcl_xlwic_7.2k_mix_seq_run_2"
path_fr = "/content/drive/MyDrive/SemEval Models b/French/xlmr_spaced_mcl_xlwic_7.2k_mix_seq_run_2"
path_ru = "/content/drive/MyDrive/SemEval Models b/Russian/xlmr_spaced_mcl_xlwic_7.2k_mix_seq_run_2"
path_zh = "/content/drive/MyDrive/SemEval Models b/Chinese/xlmr_spaced_mcl_xlwic_7.2k_mix_seq_run_2"

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 15
best_val_accuracy = 0
best_val_accuracy_ar = 0
best_val_accuracy_en = 0
best_val_accuracy_fr = 0
best_val_accuracy_ru = 0
best_val_accuracy_zh = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # if epoch_i ==0:
        #   break

        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step == int(len(train_dataloader)/4) or step == int(2*len(train_dataloader)/4) or step == int(3*len(train_dataloader)/4):
          print("")
          print("Running Validation...")

          t0 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model_bert.eval()
          model_log_reg.eval()

          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch_dev in dev_dataloader:

              with torch.no_grad():
                  b_input_ids = batch_dev[0].to(device)
                  b_attention_mask = batch_dev[1].to(device)
                  b_poses = batch_dev[2].to(device)
                  b_labels = batch_dev[3].to(device)        

                  last_layer = model_bert(b_input_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              # print(logits)
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          # print(all_dev_logits)
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[:800], all_dev_labels[:800])
          print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
          en_dev_acc = flat_accuracy_single_logit(all_dev_logits[800:], all_dev_labels[800:])


          # ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
          # zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
          # fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
          # ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
          ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])
          zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[600:800], all_dev_labels[600:800])
          fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:400], all_dev_labels[200:400])
          ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[400:600], all_dev_labels[400:600])

          print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc))
          print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))
          print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc))
          print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc))
          print("EN dev Accuracy: {0:.4f}".format(en_dev_acc))

          # Calculate the average loss over all of the batches.
          avg_val_loss = cross_entropy(all_dev_logits[:800], all_dev_labels[:800])
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)

          if(avg_val_accuracy > best_val_accuracy):
            flag = True
            best_val_accuracy = avg_val_accuracy
            best_val_preds = all_dev_logits
            torch.save(model_bert.state_dict(), os.path.join(path_mix, "xlmr_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
            np.save(os.path.join(path_mix, "best_dev_preds"), best_val_preds)

          if(ar_dev_acc > best_val_accuracy_ar):
            flag = True
            best_val_accuracy_ar = ar_dev_acc
            best_val_preds_ar = all_dev_logits
            torch.save(model_bert.state_dict(), os.path.join(path_ar, "xlmr_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_ar, "log_reg_check_best_dev"))
            np.save(os.path.join(path_ar, "best_dev_preds"), best_val_preds_ar)

          if(fr_dev_acc > best_val_accuracy_fr):
            flag = True
            best_val_accuracy_fr = fr_dev_acc
            best_val_preds_fr = all_dev_logits
            torch.save(model_bert.state_dict(), os.path.join(path_fr, "xlmr_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_fr, "log_reg_check_best_dev"))
            np.save(os.path.join(path_fr, "best_dev_preds"), best_val_preds_fr)

          if(ru_dev_acc > best_val_accuracy_ru):
            flag = True
            best_val_accuracy_ru = ru_dev_acc
            best_val_preds_ru = all_dev_logits
            torch.save(model_bert.state_dict(), os.path.join(path_ru, "xlmr_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_ru, "log_reg_check_best_dev"))
            np.save(os.path.join(path_ru, "best_dev_preds"), best_val_preds_ru)

          if(zh_dev_acc > best_val_accuracy_zh):
            flag = True
            best_val_accuracy_zh = zh_dev_acc
            best_val_preds_zh = all_dev_logits
            torch.save(model_bert.state_dict(), os.path.join(path_zh, "xlmr_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_zh, "log_reg_check_best_dev"))
            np.save(os.path.join(path_zh, "best_dev_preds"), best_val_preds_zh)

          if(en_dev_acc > best_val_accuracy_en):
            flag = True
            best_val_accuracy_en = en_dev_acc
            best_val_preds_en = all_dev_logits
            torch.save(model_bert.state_dict(), os.path.join(path_en, "xlmr_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_en, "log_reg_check_best_dev"))
            np.save(os.path.join(path_en, "best_dev_preds"), best_val_preds_en)
          
          print("  Validation Loss: {0:.3f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))

          model_bert.train()
          model_log_reg.train()

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        optimizer.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        last_layer = model_bert(b_input_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch_dev in dev_dataloader:

        with torch.no_grad():
            b_input_ids = batch_dev[0].to(device)
            b_attention_mask = batch_dev[1].to(device)
            b_poses = batch_dev[2].to(device)
            b_labels = batch_dev[3].to(device)        

            last_layer = model_bert(b_input_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[:800], all_dev_labels[:800])
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    en_dev_acc = flat_accuracy_single_logit(all_dev_logits[800:], all_dev_labels[800:])


    # ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
    # zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
    # fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
    # ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
    ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])
    zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[600:800], all_dev_labels[600:800])
    fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:400], all_dev_labels[200:400])
    ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[400:600], all_dev_labels[400:600])

    print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc))
    print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))
    print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc))
    print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc))
    print("EN dev Accuracy: {0:.4f}".format(en_dev_acc))

    # Calculate the average loss over all of the batches.
    avg_val_loss = cross_entropy(all_dev_logits[:800], all_dev_labels[:800])
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      flag = True
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_mix, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
      np.save(os.path.join(path_mix, "best_dev_preds"), best_val_preds)

    if(ar_dev_acc > best_val_accuracy_ar):
      flag = True
      best_val_accuracy_ar = ar_dev_acc
      best_val_preds_ar = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_ar, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_ar, "log_reg_check_best_dev"))
      np.save(os.path.join(path_ar, "best_dev_preds"), best_val_preds_ar)

    if(fr_dev_acc > best_val_accuracy_fr):
      flag = True
      best_val_accuracy_fr = fr_dev_acc
      best_val_preds_fr = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_fr, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_fr, "log_reg_check_best_dev"))
      np.save(os.path.join(path_fr, "best_dev_preds"), best_val_preds_fr)

    if(ru_dev_acc > best_val_accuracy_ru):
      flag = True
      best_val_accuracy_ru = ru_dev_acc
      best_val_preds_ru = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_ru, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_ru, "log_reg_check_best_dev"))
      np.save(os.path.join(path_ru, "best_dev_preds"), best_val_preds_ru)

    if(zh_dev_acc > best_val_accuracy_zh):
      flag = True
      best_val_accuracy_zh = zh_dev_acc
      best_val_preds_zh = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_zh, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_zh, "log_reg_check_best_dev"))
      np.save(os.path.join(path_zh, "best_dev_preds"), best_val_preds_zh)

    if(en_dev_acc > best_val_accuracy_en):
      flag = True
      best_val_accuracy_en = en_dev_acc
      best_val_preds_en = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_en, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_en, "log_reg_check_best_dev"))
      np.save(os.path.join(path_en, "best_dev_preds"), best_val_preds_en)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))


        # Do same on test data
    # Tracking variables 
    # total_eval_accuracy = 0
    # total_eval_loss = 0
    # nb_eval_steps = 0

    # t_labels = []
    # t_preds = []

    # # Evaluate data for one epoch
    # for batch in test_dataloader:
        
    #     with torch.no_grad():
    #         b_input_ids = batch[0].to(device)
    #         b_attention_mask = batch[1].to(device)
    #         b_poses = batch[2].to(device)
    #         b_labels = batch[3].to(device)        

    #         last_layer = model_bert(b_input_ids, b_attention_mask)
    #         b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
    #         gathered_activations = torch.gather(last_layer, 1, b_poses)
    #         logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
    #         loss = loss_fn(logits, b_labels.type_as(logits))
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.cpu().detach().numpy()
    #     label_ids = b_labels.cpu().numpy()

    #     t_labels.append(label_ids)
    #     t_preds.append(logits)
        

    # all_dev_labels = np.concatenate(t_labels, axis=0)
    # all_dev_logits = np.concatenate(t_preds, axis=0)
    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[800:], all_dev_labels[800:])
    # print("Test Accuracy: {0:.4f}".format(avg_val_accuracy))
    # en_test_acc = flat_accuracy_single_logit(all_dev_logits[:800], all_dev_labels[:800])
    # actual_val = (actual_val + 80*en_test_acc)/100
    # print("Actual En Val Accuracy: {0:.4f}".format(actual_val))
    # ru_test_acc = flat_accuracy_single_logit(all_dev_logits[800:1000], all_dev_labels[800:1000])
    # zh_test_acc = flat_accuracy_single_logit(all_dev_logits[1000:1200], all_dev_labels[1000:1200])
    # ar_test_acc = flat_accuracy_single_logit(all_dev_logits[1200:1400], all_dev_labels[1200:1400])
    # fr_test_acc = flat_accuracy_single_logit(all_dev_logits[1400:1600], all_dev_labels[1400:1600])
    # print("EN Test Accuracy: {0:.4f}".format(en_test_acc))
    # print("RU Test Accuracy: {0:.4f}".format(ru_test_acc))
    # print("ZH Test Accuracy: {0:.4f}".format(zh_test_acc))
    # print("AR Test Accuracy: {0:.4f}".format(ar_test_acc))
    # print("FR Test Accuracy: {0:.4f}".format(fr_test_acc))

    # if flag == True:
    #   np.save(os.path.join(path_mix, "test_preds_at_best_val"), all_dev_logits)
    #   flag = False

    # # if(actual_val > t_best_val_accuracy):
    # #   t_best_val_accuracy = actual_val
    # #   t_best_val_preds = all_dev_logits
    # #   torch.save(model_bert.state_dict(), os.path.join(path_this, "xlmr_check_best_test"))
    # #   torch.save(model_log_reg.state_dict(), os.path.join(path_this, "log_reg_check_best_test"))
    # #   np.save(os.path.join(path_this, "best_test_preds"), t_best_val_preds)

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(test_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Test Avg Loss: {0:.3f}".format(avg_val_loss))
    # print("  Testing Validation took: {:}".format(validation_time))





print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 15 ========
Training...
  Batch   100  of  2,432.    Elapsed: 0:00:44.
  Batch   200  of  2,432.    Elapsed: 0:01:27.
  Batch   300  of  2,432.    Elapsed: 0:02:11.
  Batch   400  of  2,432.    Elapsed: 0:02:54.
  Batch   500  of  2,432.    Elapsed: 0:03:38.
  Batch   600  of  2,432.    Elapsed: 0:04:22.

Running Validation...
Validation Accuracy: 0.8013
AR dev Accuracy: 0.7650 
FR dev Accuracy: 0.8600 
RU dev Accuracy: 0.8300 
ZH dev Accuracy: 0.7500 
EN dev Accuracy: 0.7790
  Validation Loss: 0.478
  Validation took: 0:00:18
  Batch   700  of  2,432.    Elapsed: 0:04:05.
  Batch   800  of  2,432.    Elapsed: 0:04:49.
  Batch   900  of  2,432.    Elapsed: 0:05:32.
  Batch 1,000  of  2,432.    Elapsed: 0:06:16.
  Batch 1,100  of  2,432.    Elapsed: 0:07:00.
  Batch 1,200  of  2,432.    Elapsed: 0:07:43.

Running Validation...
Validation Accuracy: 0.7950
AR dev Accuracy: 0.8050 
FR dev Accuracy: 0.8500 
RU dev Accuracy: 0.8300 
ZH dev Accuracy: 0.6950 
EN dev Accurac

In [ ]:
print(best_val_accuracy) # same as none
print(best_val_accuracy_ar)
print(best_val_accuracy_en)
print(best_val_accuracy_fr)
print(best_val_accuracy_ru)
print(best_val_accuracy_zh)

0.87
0.875
0.889
0.91
0.925
0.82
